In [239]:
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import plotly.graph_objects as go

In [240]:
response = requests.get('https://api.worldtradingdata.com/api/v1/history?symbol=SAN.MC&api_token=UwX3ZumXmZiEtZfdnXZKIXAU743QiGU3Lhf8SD87Tu7lzVORrl74twgQiZjc&date_from=2010-01-01')
san_dict = json.loads(response.text)

In [241]:
print(san_dict)

{'name': 'SAN.MC', 'history': {'2020-03-20': {'open': '2.12', 'close': '2.11', 'high': '2.16', 'low': '2.01', 'volume': '168574910'}, '2020-03-19': {'open': '2.11', 'close': '2.04', 'high': '2.13', 'low': '2.00', 'volume': '114341780'}, '2020-03-18': {'open': '2.13', 'close': '2.04', 'high': '2.18', 'low': '2.00', 'volume': '126859015'}, '2020-03-17': {'open': '2.22', 'close': '2.17', 'high': '2.22', 'low': '1.96', 'volume': '148558648'}, '2020-03-16': {'open': '2.19', 'close': '2.02', 'high': '2.20', 'low': '1.93', 'volume': '198143573'}, '2020-03-13': {'open': '2.37', 'close': '2.26', 'high': '2.61', 'low': '2.19', 'volume': '156408177'}, '2020-03-12': {'open': '2.62', 'close': '2.28', 'high': '2.63', 'low': '2.27', 'volume': '229408705'}, '2020-03-11': {'open': '2.70', 'close': '2.74', 'high': '2.85', 'low': '2.68', 'volume': '124662252'}, '2020-03-10': {'open': '2.75', 'close': '2.65', 'high': '2.89', 'low': '2.62', 'volume': '134400193'}, '2020-03-09': {'open': '2.90', 'close': '2

In [242]:
df_san = pd.DataFrame.from_dict(san_dict['history'], orient='index')
df_san.head(10)

,open,close,high,low,volume
2020-03-20,2.12,2.11,2.16,2.01,168574910
2020-03-19,2.11,2.04,2.13,2.00,114341780
2020-03-18,2.13,2.04,2.18,2.00,126859015
2020-03-17,2.22,2.17,2.22,1.96,148558648
2020-03-16,2.19,2.02,2.20,1.93,198143573
2020-03-13,2.37,2.26,2.61,2.19,156408177
2020-03-12,2.62,2.28,2.63,2.27,229408705
2020-03-11,2.70,2.74,2.85,2.68,124662252
2020-03-10,2.75,2.65,2.89,2.62,134400193
2020-03-09,2.90,2.68,2.90,2.63,198998279


In [243]:
# Las peticiones al api son limitadas, lo que hacemos es traernos todos los datos de todos
# los valores del ibex35 y los insertamos en una base de datos. Los datos históricos son a cierre de mercado.
# Le añadimos la hora para cuando carguemos los datos en realtime que si viene la hora y minutos
#df_san.index = pd.to_datetime(df_san.index + ' ' + '17:36')

In [244]:
df_san.head(10)

,open,close,high,low,volume
2020-03-20,2.12,2.11,2.16,2.01,168574910
2020-03-19,2.11,2.04,2.13,2.00,114341780
2020-03-18,2.13,2.04,2.18,2.00,126859015
2020-03-17,2.22,2.17,2.22,1.96,148558648
2020-03-16,2.19,2.02,2.20,1.93,198143573
2020-03-13,2.37,2.26,2.61,2.19,156408177
2020-03-12,2.62,2.28,2.63,2.27,229408705
2020-03-11,2.70,2.74,2.85,2.68,124662252
2020-03-10,2.75,2.65,2.89,2.62,134400193
2020-03-09,2.90,2.68,2.90,2.63,198998279


In [245]:
#df_close_san = df_san.loc[df_san.index >= '2019-03-01']
#df_close_san = df_san[['close']].astype(float)
#df_close_san.sort_index(inplace=True, ascending=True)
#df_close_san = df_close_san.reset_index()

In [246]:
# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_san.index, y=df_san.close))

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.update_layout(
    title="Santander Price History",
    xaxis_title="Date",
    yaxis_title="Price",
    width=1024,
    height=600,
    font=dict(
        family="Courier New, monospace",
        size=15,
        color="#7f7f7f"
    )
)
fig.show()